In [1]:
import pandas as pd
import numpy as np
import tensorflow

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedStratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report

seed = 2000
np.random.seed(seed)
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import BatchNormalization, Dropout, Flatten, Dense, Embedding
from tensorflow.keras.layers import LSTM, Conv1D, MaxPooling1D
from tensorflow.keras.preprocessing import sequence

from gensim.models import Word2Vec
from gensim.parsing.porter import PorterStemmer

import nltk
from nltk.tokenize import TweetTokenizer

import warnings
warnings.filterwarnings("ignore")

2023-01-04 12:34:13.092767: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
train_sub1_df = pd.read_csv('/Users/nitanshjain/Documents/Miscellaneous/SemEval/semeval2023task3/preprocessed_data/subtask1/train_subtask_1.csv')
print(train_sub1_df.shape)
train_sub1_df.head()
print(train_sub1_df.genre.value_counts())

(433, 10)
opinion      382
reporting     41
satire        10
Name: genre, dtype: int64


In [3]:
le = LabelEncoder()
train_sub1_df['genre'] = le.fit_transform(train_sub1_df['genre'])
print(train_sub1_df.genre.value_counts())
train_sub1_df.head()

0    382
1     41
2     10
Name: genre, dtype: int64


,id,genre,headlines,articles,preprocessed_headlines,pos_tags_headlines,er_tags_headlines,preprocessed_articles,pos_tags_articles,er_tags_articles
0,833042063,2,Chelsea Handler Admits She’s ‘Very Sexually At...,Far-left comedienne Chelsea Handler has admitt...,chelsea handler admit she s very sexually attr...,"[(Chelsea, 'NNP'), (Handler, 'NNP'), (admit, '...","[(Chelsea Handler, 'PERSON', 380), (Robert Mue...",far leave comedienne chelsea handler have admi...,"[(far, 'RB'), (leave, 'VB'), (comedienne, 'NNP...","[(Chelsea Handler, 'PERSON', 380), (FBI Specia..."
1,832959523,2,How Theresa May Botched\n,Those were the times…\nThe Times page 1 is of ...,how theresa may botch,"[(how, 'WRB'), (Theresa, 'NNP'), (May, 'NNP'),...","[(Theresa, 'GPE', 384), (May, 'DATE', 391)]",those be the time the times page 1 be of janua...,"[(those, 'DT'), (be, 'VBP'), (the, 'DT'), (tim...","[(Times, 'ORG', 383), (1, 'CARDINAL', 397), (J..."
2,833039623,2,Robert Mueller III Rests His Case—Dems NEVER W...,Carload of crazies headed to the White House w...,robert mueller iii rest his case dems never will,"[(Robert, 'NNP'), (Mueller, 'NNP'), (III, 'NNP...","[(Robert Mueller III, 'PERSON', 380), (Dems, '...",carload of crazy head to the white house want ...,"[(Carload, 'NNP'), (of, 'IN'), (crazy, 'JJ'), ...","[(the White House, 'ORG', 383), (Barack Obama,..."
3,833032367,2,Robert Mueller Not Recommending Any More Indic...,"But of course, this makes no difference to the...",robert mueller not recommend any more indictment,"[(Robert, 'NNP'), (Mueller, 'NNP'), (not, 'RB'...","[(Robert Mueller, 'PERSON', 380)]",but of course this make no difference to the p...,"[(but, 'CC'), (of, 'IN'), (course, 'NN'), (thi...","[(the New York Times, 'ORG', 383), (late Frida..."
4,814777937,2,The Far Right Is Trying to Co-opt the Yellow V...,"This weekend in Manchester, England, Yellow Ve...",the far right be try to co opt the yellow vests,"[(the, 'DT'), (Far, 'NNP'), (right, 'NN'), (be...",[],this weekend in manchester england yellow vest...,"[(this, 'DT'), (weekend, 'NN'), (in, 'IN'), (M...","[(this weekend, 'DATE', 391), (Manchester, 'GP..."


In [4]:
def creating_tokens(df):
    tokens = list()
    tokenizer = TweetTokenizer()
    
    for tweets in df.loc[:, 'preprocessed_articles']:
        # print(len(tokenizer.tokenize(tweets)))
        tokens.append(tokenizer.tokenize(tweets))
    
    df['tokens_articles'] = tokens
    
    porter_stemmer = PorterStemmer()
    # Get the stemmed_tokens
    df['stemmed_tokens_articles'] = [[porter_stemmer.stem(word) for word in tokens] for tokens in df['tokens_articles']]
    
    return df

In [5]:
train_sub1_df = creating_tokens(train_sub1_df)
x_tokens = train_sub1_df.stemmed_tokens_articles
x_tokens = x_tokens.to_frame().reset_index()

y = train_sub1_df.genre

OUTPUT_FOLDER = '/Users/nitanshjain/Documents/Miscellaneous/SemEval/semeval2023task3/codefiles/subtask1/word2vec/skipgram/articles/'

tokens = pd.Series(train_sub1_df.stemmed_tokens_articles).values
# print(tokens)
word2vec_model_file = OUTPUT_FOLDER + 'word2vec_subtask1_' + str(200) + '.model'

w2v_model = Word2Vec(tokens, min_count=1, vector_size=200, window=5, workers=4, sg=2)
w2v_model.train(tokens, epochs=10, total_examples=len(tokens))
w2v_model.save(word2vec_model_file)

In [6]:
def create_file(create_file, model_file, x):
    sg_w2v_model = Word2Vec.load(model_file)
    
    with open(create_file, 'w+') as word2vec_file:
        for index, row in x.iterrows():
            model_vector = (np.mean([sg_w2v_model.wv[token] for token in row['stemmed_tokens_articles']], axis=0)).tolist()
            if index == 0:
                header = ",".join(str(ele) for ele in range(200))
                word2vec_file.write(header)
                word2vec_file.write("\n")
            
            if type(model_vector) is list:
                line1 = ",".join( [str(vector_element) for vector_element in model_vector] )
            else:
                line1 = ",".join([str(0) for i in range(200)])
            word2vec_file.write(line1)
            word2vec_file.write('\n')
    
    df = pd.read_csv(create_file)
    return df
        

In [7]:
word2vec_train_filename = OUTPUT_FOLDER + 'word2vec_subtask1_train_' + str(200) + '.csv'
word2vec_train_df = create_file(word2vec_train_filename, word2vec_model_file, x_tokens)
print(word2vec_train_df.shape)
word2vec_train_df.head()

(433, 200)


,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,-0.002776,-0.092952,-0.192130,0.093177,0.103150,-0.183182,0.119462,0.228124,-0.056931,-0.006100,...,0.190338,0.155121,-0.197320,-0.107869,0.100324,0.146585,-0.091133,-0.074788,0.110376,0.038289
1,-0.001653,-0.103344,-0.169363,0.092336,0.053393,-0.137953,0.100718,0.237850,-0.032418,0.061889,...,0.205370,0.106118,-0.174401,-0.075590,0.084942,0.172072,-0.051089,-0.097576,0.100182,-0.020281
2,0.015665,-0.078747,-0.173598,0.103440,0.086101,-0.120865,0.112737,0.254691,-0.053829,0.037047,...,0.180553,0.130849,-0.163068,-0.094939,0.112312,0.156304,-0.097507,-0.108177,0.086815,-0.011156
3,-0.017504,-0.128308,-0.196425,0.133985,0.074240,-0.159517,0.088086,0.263130,-0.087183,0.027116,...,0.143575,0.116591,-0.171830,-0.148698,0.077723,0.157007,-0.112070,-0.123995,0.081044,0.066010
4,0.057433,-0.041737,-0.116973,0.049703,0.046053,-0.093307,0.120918,0.204406,-0.091702,0.026233,...,0.161535,0.091975,-0.103320,-0.174833,0.074652,0.114995,-0.025193,-0.106041,0.057429,-0.014550


In [8]:
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()

x_scaled = pd.DataFrame(mm.fit_transform(word2vec_train_df))
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=seed)
x_scaled.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.300885,0.440033,0.265790,0.411694,0.713263,0.269513,0.777401,0.359550,0.460269,0.267389,...,0.311952,0.862587,0.262946,0.647703,0.457528,0.566722,0.563569,0.909789,0.773774,0.532015
1,0.306436,0.390284,0.367908,0.406329,0.496248,0.521230,0.698736,0.406387,0.568091,0.622614,...,0.369484,0.686766,0.361173,0.809579,0.374963,0.673587,0.769910,0.766139,0.724148,0.274709
2,0.392018,0.508042,0.348911,0.477175,0.638905,0.616334,0.749179,0.487493,0.473915,0.492818,...,0.274501,0.775499,0.409743,0.712543,0.521878,0.607476,0.530728,0.699312,0.659074,0.314796
3,0.228102,0.270768,0.246526,0.672060,0.587172,0.401220,0.645723,0.528132,0.327207,0.440934,...,0.132971,0.724341,0.372190,0.442950,0.336210,0.610420,0.455687,0.599600,0.630980,0.653796
4,0.598434,0.685224,0.602897,0.134325,0.464234,0.769701,0.783512,0.245329,0.307329,0.436322,...,0.201713,0.636020,0.665808,0.311881,0.319727,0.434270,0.903350,0.712775,0.516017,0.299886


# KNN

In [9]:
from sklearn.neighbors import KNeighborsClassifier
knn_params = {
    'n_neighbors':range(1,15),
    'weights':['uniform', 'distance']
}

knn = KNeighborsClassifier()
clf = GridSearchCV(knn, knn_params, cv=cv)
clf.fit(x_scaled, y)

print('Best Accuracy Score ', clf.best_score_)
print('Best Parameters', clf.best_params_)

Best Accuracy Score  0.888372093023256
Best Parameters {'n_neighbors': 4, 'weights': 'uniform'}


# Logistic Regression

In [10]:
from sklearn.linear_model import LogisticRegression
lr_params = {
    'C':np.logspace(-2,2,7), 
    'penalty':['l1', 'l2', 'elasticnet', None]
}

lr = LogisticRegression()
clf = GridSearchCV(lr, lr_params, cv=cv)
clf.fit(x_scaled, y)

print('Best Accuracy Score ', clf.best_score_)
print('Best Parameters', clf.best_params_)

Best Accuracy Score  0.8853241719520792
Best Parameters {'C': 0.21544346900318834, 'penalty': 'l2'}


# Decison Tree

In [11]:
from sklearn.tree import DecisionTreeClassifier

dt_params = {
    'max_depth':range(2,15),
    'criterion':['gini', 'entropy', 'log_loss'],
    'splitter':['best', 'random']
}

dt = DecisionTreeClassifier()
clf = GridSearchCV(dt, dt_params, cv=cv)
clf.fit(x_scaled, y)

print('Best Accuracy Score ', clf.best_score_)
print('Best Parameters', clf.best_params_)

Best Accuracy Score  0.8845489781536297
Best Parameters {'criterion': 'entropy', 'max_depth': 2, 'splitter': 'random'}


# Random Forest Classifier

In [12]:
from sklearn.ensemble import RandomForestClassifier

rfc_params = {
    'max_depth':range(3,15),
    'criterion':['gini', 'entropy', 'log_loss'],
}

rfc = RandomForestClassifier()
clf = GridSearchCV(rfc, rfc_params, cv=cv)
clf.fit(x_scaled, y)

print('Best Accuracy Score ', clf.best_score_)
print('Best Parameters', clf.best_params_)

Best Accuracy Score  0.8891825229034533
Best Parameters {'criterion': 'gini', 'max_depth': 8}


# Multinomial Naive Bayes

In [13]:
%%time
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
mnb.fit(x_scaled, y)

print('Train Accuracy Score', mnb.score(x_scaled, y))

Train Accuracy Score 0.8868360277136259
CPU times: user 11.2 ms, sys: 8.87 ms, total: 20 ms
Wall time: 18.1 ms


# SVM

In [14]:
from sklearn.svm import SVC

svc_params = {
    'kernel':['linear', 'poly', 'rbf', 'sigmoid'],
    'degree':range(2,5)
}

svc = SVC()
clf = GridSearchCV(svc, svc_params, cv=cv)
clf.fit(x_scaled, y)

print('Best Accuracy Score ', clf.best_score_)
print('Best Parameters', clf.best_params_)

Best Accuracy Score  0.8822410147991546
Best Parameters {'degree': 2, 'kernel': 'sigmoid'}


# Ada-Boosting

In [15]:
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier()
ada.fit(x_scaled, y)

print('Train Accuracy Score', ada.score(x_scaled, y))

Train Accuracy Score 0.7875288683602771


# LSTM

In [ ]:
x_scaled = x_scaled[:,:,None]

In [ ]:
from keras import Sequential
from keras.layers import LSTM

model_lstm = Sequential()

model_lstm.add(LSTM(64, input_shape = x_scaled.shape[1:], return_sequences = True))
model_lstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_lstm.add(Dropout(rate=0.8))

model_lstm.add(LSTM(32, return_sequences=True))
model_lstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_lstm.add(Dropout(rate=0.5))

model_lstm.add(LSTM(16, return_sequences=True))
model_lstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_lstm.add(Dropout(rate=0.5))

model_lstm.add(LSTM(8, return_sequences=True))
model_lstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_lstm.add(Dropout(rate=0.5))

model_lstm.add(LSTM(1, return_sequences=True, name='output'))
model_lstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_lstm.add(Dropout(rate=0.5))

model_lstm.add(Dense(64, activation = 'relu'))
model_lstm.add(Flatten())
model_lstm.add(Dense(1, activation='softmax'))

model_lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])

model_lstm.summary()

2022-12-27 14:13:43.797353: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 11720, 64)         16896     
                                                                 
 dropout (Dropout)           (None, 11720, 64)         0         
                                                                 
 lstm_1 (LSTM)               (None, 11720, 32)         12416     
                                                                 
 dropout_1 (Dropout)         (None, 11720, 32)         0         
                                                                 
 lstm_2 (LSTM)               (None, 11720, 16)         3136      
                                                                 
 dropout_2 (Dropout)         (None, 11720, 16)         0         
                                                                 
 lstm_3 (LSTM)               (None, 11720, 8)          8

In [ ]:
batch_size = 32

model_lstm.fit(x_scaled, y,
            batch_size=batch_size,
            epochs=10,
            shuffle=True)

Epoch 1/5
4/7 [================>.............] - ETA: 2:36 - loss: 0.6169 - accuracy: 0.0859

# CNN LSTM

In [ ]:
from keras import Sequential
from keras.layers import LSTM

model_clstm  =  Sequential()

model_clstm.add(Conv1D(32, (3), padding = 'same', activation = 'relu', input_shape = x_scaled.shape[1:]))
model_clstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_clstm.add(Dropout(rate = 0.8))

model_clstm.add(Conv1D(16, (3), padding = 'same', activation = 'relu', input_shape = x_scaled.shape[1:]))
model_clstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_clstm.add(Dropout(rate = 0.8))

model_clstm.add(Conv1D(8, (3), padding = 'same', activation = 'relu', input_shape = x_scaled.shape[1:]))
model_clstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_clstm.add(Dropout(rate = 0.8))

model_clstm.add(LSTM(32, return_sequences = True))
model_clstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_clstm.add(Dropout(rate = 0.5))

model_clstm.add(LSTM(16, return_sequences = True))
model_clstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_clstm.add(Dropout(rate = 0.5))

model_clstm.add(LSTM(1, return_sequences = True))
model_clstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_clstm.add(Dropout(rate = 0.5))

model_clstm.add(Dense(64, activation = 'relu'))
model_clstm.add(Flatten())
model_clstm.add(Dense(1, activation = 'softmax'))

model_clstm.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model_clstm.summary()

In [ ]:
batch_size = 32

model_clstm.fit(x_scaled, y,
            batch_size=batch_size,
            epochs=10,
            shuffle=True)

# Bi-LSTM

In [ ]:
from keras import Sequential
from keras.layers import LSTM, Embedding, Bidirectional


model_bilstm  =  Sequential()

model_bilstm.add(Bidirectional(LSTM(32, input_shape = x_scaled.shape[1:], return_sequences=True)))
model_bilstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_bilstm.add(Dropout(rate = 0.5))

model_bilstm.add(Bidirectional(LSTM(16, return_sequences=True)))
model_bilstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_bilstm.add(Dropout(rate = 0.5))

model_bilstm.add(Bidirectional(LSTM(1, return_sequences=True)))
model_bilstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_bilstm.add(Dropout(rate = 0.5))

model_bilstm.add(Dense(64, activation = 'relu'))
model_bilstm.add(Flatten())
model_bilstm.add(Dense(1, activation = 'softmax'))

model_bilstm.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model_bilstm.summary()

In [ ]:
batch_size = 32

model_bilstm.fit(x_scaled, y,
            batch_size=batch_size,
            epochs=10,
            shuffle=True)